In [1]:
# Importing the required libraries
from IPython.display import display
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch.autograd import Variable
from torch_geometric.nn import GCNConv
from torch.utils.data import TensorDataset, DataLoader
import os
import glob
from sklearn.preprocessing import MinMaxScaler
import time

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

In [2]:
class Net(torch.nn.Module):
    def __init__(self, d_feat, edge_indexs, nnode):
        super(Net, self).__init__()
        self.edge_index = edge_indexs
        self.nnode = nnode
        self.conv1 = GCNConv(d_feat, 32, cached=True,
                             normalize= True)
        self.conv2 = GCNConv(32, 32, cached=True,
                             normalize=True)
        self.conv3 = GCNConv(32, 8, cached=True,
                             normalize=True)
        self.linears = torch.nn.ModuleList([torch.nn.Linear(8, 1) for _ in range(nnode)])


    def forward(self, x):
        x, edge_index, edge_weight = x, self.edge_index, None
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.conv2(x, edge_index, edge_weight))
        x = F.relu(self.conv3(x, edge_index, edge_weight))
        
        out = [self.linears[i](x[i, :]) for i in range(self.nnode)]

        return out

## Processing the file to input into the model

In [3]:
# Specifying the path of file
path= r"F:\GAMES Research\GAT\GAT\DOE_Q1\L2\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['COAST.csv', 'EAST.csv', 'FWEST.csv', 'NCENT.csv', 'NORTH.csv', 'SCENT.csv', 'SOUTH.csv', 'WEST.csv']


In [4]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[0]))

,Net,Temprature
0,9844.200268,6.8
1,9653.456269,7.1
2,9502.299659,7.1
3,9474.478036,7.0
4,9554.370489,6.5
...,...,...
43818,11109.257856,14.8
43819,10846.500062,14.3
43820,10592.539874,14.1
43821,10304.624870,13.7


## Processing the load data into train-test split

In [5]:
# Fetching and concatenating the data
Load_DS = pd.concat([pd.read_csv(os.path.join(path, fileName), usecols= ["Net"]) for fileName in fileList], axis= 1)
Load_DS.columns = [i.removesuffix('.csv') for i in fileList]
Load_DS = Load_DS.add_prefix("LOAD_")

Load_DS

,LOAD_COAST,LOAD_EAST,LOAD_FWEST,LOAD_NCENT,LOAD_NORTH,LOAD_SCENT,LOAD_SOUTH,LOAD_WEST
0,9844.200268,1350.784678,2078.060021,13640.024978,901.770195,6731.301663,3607.904503,1470.814721
1,9653.456269,1333.434943,2057.993265,13425.121941,890.150373,6602.935629,3609.619438,1440.832944
2,9502.299659,1323.908498,2039.019542,13286.070909,880.926639,6530.663273,3573.158363,1430.495044
3,9474.478036,1325.951405,2027.431312,13310.892590,883.990576,6503.367925,3538.420320,1423.806348
4,9554.370489,1339.675290,2026.809889,13423.345503,894.320303,6603.227872,3542.905394,1428.180823
...,...,...,...,...,...,...,...,...
43818,11109.257856,1417.306742,3637.456602,13164.660616,825.761799,6453.697286,3181.359858,1290.258402
43819,10846.500062,1411.761157,3639.019312,12928.911509,822.190573,6175.763027,3044.648834,1276.748646
43820,10592.539874,1403.990584,3639.463899,12716.274141,819.394012,5968.287603,2930.563230,1260.087906
43821,10304.624870,1387.051521,3630.184859,12413.090126,804.714775,5769.745220,2829.531899,1238.412254


In [6]:
Load_DS[Load_DS.isna().any(axis=1)]

,LOAD_COAST,LOAD_EAST,LOAD_FWEST,LOAD_NCENT,LOAD_NORTH,LOAD_SCENT,LOAD_SOUTH,LOAD_WEST


In [7]:
# Creating 1-day lag loads
Load_lag_1 = Load_DS.shift(24).fillna(0)
Load_lag_1.columns = [i.removeprefix('LOAD_') for i in Load_lag_1.columns]
Load_lag_1 = Load_lag_1.add_prefix("LAG1_")
Load_lag_1

,LAG1_COAST,LAG1_EAST,LAG1_FWEST,LAG1_NCENT,LAG1_NORTH,LAG1_SCENT,LAG1_SOUTH,LAG1_WEST
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...
43818,11383.135155,1501.958142,3785.625918,14134.787888,870.666623,6702.451162,3171.171394,1366.615020
43819,11373.392115,1512.317113,3819.840998,14219.251639,878.711519,6747.748182,3162.433299,1393.605790
43820,11207.914145,1505.305483,3835.426633,14082.197457,868.522154,6694.905174,3115.097385,1406.001524
43821,10894.781874,1458.874421,3823.477228,13637.973450,847.317385,6539.604065,3022.212876,1377.215087


In [8]:
# Creating 7-day lag loads
Load_lag_7 = Load_DS.shift(24*7).fillna(0)
Load_lag_7.columns = [i.removeprefix('LOAD_') for i in Load_lag_7.columns]
Load_lag_7 = Load_lag_7.add_prefix("LAG7_")
Load_lag_7

,LAG7_COAST,LAG7_EAST,LAG7_FWEST,LAG7_NCENT,LAG7_NORTH,LAG7_SCENT,LAG7_SOUTH,LAG7_WEST
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...
43818,10523.664434,1222.397590,3649.932706,11434.247698,720.820842,5780.380475,3033.234778,1153.226635
43819,10335.992990,1237.349389,3640.453277,11231.263595,719.864419,5693.923783,2913.452696,1149.420623
43820,10144.751404,1236.197626,3624.437253,11031.084524,714.949676,5624.696898,2828.170796,1139.002281
43821,9876.787820,1216.187797,3619.528769,10749.333909,698.896222,5512.343425,2778.351469,1124.153791


In [9]:
# Setting the train-test split
ratio = 0.2
Num_test, Num_train = int(len(Load_DS) * ratio), len(Load_DS) - int(len(Load_DS) * ratio)

In [10]:
# Scaling the load data wrt WEST
mmScaler_load = MinMaxScaler()

# Splitting the data into train and test [LOAD]
Load_train, Load_test = Load_DS[:Num_train], Load_DS[Num_train:]
print("Raw load - Train: ")
display(Load_train.head(2))
print("\n")

# Splitting the data into train and test [LAG 1]
Load_Lag1_train, Load_Lag1_test = Load_lag_1[:Num_train], Load_lag_1[Num_train:]
print("Raw lag 1 load - Train: ")
display(Load_Lag1_train.head(2))
print("\n")

# Splitting the data into train and test [LAG 7]
Load_Lag7_train, Load_Lag7_test = Load_lag_7[:Num_train], Load_lag_7[Num_train:]
print("Raw lag 7 load - Train: ")
display(Load_Lag7_train.head(2))
print("\n")

Raw load - Train: 


,LOAD_COAST,LOAD_EAST,LOAD_FWEST,LOAD_NCENT,LOAD_NORTH,LOAD_SCENT,LOAD_SOUTH,LOAD_WEST
0,9844.200268,1350.784678,2078.060021,13640.024978,901.770195,6731.301663,3607.904503,1470.814721
1,9653.456269,1333.434943,2057.993265,13425.121941,890.150373,6602.935629,3609.619438,1440.832944




Raw lag 1 load - Train: 


,LAG1_COAST,LAG1_EAST,LAG1_FWEST,LAG1_NCENT,LAG1_NORTH,LAG1_SCENT,LAG1_SOUTH,LAG1_WEST
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0




Raw lag 7 load - Train: 


,LAG7_COAST,LAG7_EAST,LAG7_FWEST,LAG7_NCENT,LAG7_NORTH,LAG7_SCENT,LAG7_SOUTH,LAG7_WEST
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Scaling the data using mix-max scaler [TRAINING]
Load_train = mmScaler_load.fit_transform(Load_train)
print("Scaled load - Train: ")
display(Load_train)

Load_Lag1_train = mmScaler_load.transform(Load_Lag1_train)
print("Scaled lag 1 load - Train: ")
display(Load_Lag1_train)

Load_Lag7_train = mmScaler_load.transform(Load_Lag7_train)
print("Scaled lag 7 load - Train: ")
display(Load_Lag7_train)

Scaled load - Train: 


array([[0.19856039, 0.30747691, 0.29986234, ..., 0.29388401, 0.40075164,
        0.54746011],
       [0.18389753, 0.29801661, 0.29095308, ..., 0.28169351, 0.40115179,
        0.52531792],
       [0.17227783, 0.29282212, 0.28252911, ..., 0.27483003, 0.39264419,
        0.51768315],
       ...,
       [0.27123435, 0.27969052, 0.70051389, ..., 0.2438984 , 0.29417073,
        0.31301885],
       [0.29289962, 0.31276503, 0.74330209, ..., 0.28382077, 0.34106047,
        0.38335221],
       [0.2755846 , 0.29403583, 0.74936549, ..., 0.27174927, 0.32695204,
        0.39355169]])

Scaled lag 1 load - Train: 


array([[-0.55818217, -0.42906604, -0.62275662, ..., -0.34536602,
        -0.4410945 , -0.53876871],
       [-0.55818217, -0.42906604, -0.62275662, ..., -0.34536602,
        -0.4410945 , -0.53876871],
       [-0.55818217, -0.42906604, -0.62275662, ..., -0.34536602,
        -0.4410945 , -0.53876871],
       ...,
       [ 0.30665217,  0.39624262,  0.77351347, ...,  0.38824738,
         0.40656853,  0.48944813],
       [ 0.32098894,  0.42290809,  0.81293524, ...,  0.40006934,
         0.44338406,  0.52088999],
       [ 0.31391909,  0.39200483,  0.82140623, ...,  0.39591403,
         0.44660498,  0.52418617]])

Scaled lag 7 load - Train: 


array([[-0.55818217, -0.42906604, -0.62275662, ..., -0.34536602,
        -0.4410945 , -0.53876871],
       [-0.55818217, -0.42906604, -0.62275662, ..., -0.34536602,
        -0.4410945 , -0.53876871],
       [-0.55818217, -0.42906604, -0.62275662, ..., -0.34536602,
        -0.4410945 , -0.53876871],
       ...,
       [ 0.22943945,  0.22151142,  0.62590842, ...,  0.19902454,
         0.26151862,  0.27595612],
       [ 0.231143  ,  0.23979617,  0.64939785, ...,  0.22040623,
         0.27810459,  0.31329784],
       [ 0.21343718,  0.23971091,  0.65105144, ...,  0.20553165,
         0.24673204,  0.31714201]])

In [12]:
# Scaling the data using mix-max scaler [TESTING]
Load_test = mmScaler_load.transform(Load_test)
print("Scaled load - Test: ")
display(Load_test)

Load_Lag1_test = mmScaler_load.transform(Load_Lag1_test)
print("Scaled lag 1 load - Test: ")
display(Load_Lag1_test)

Load_Lag7_test = mmScaler_load.transform(Load_Lag7_test)
print("Scaled lag 7 load - Test: ")
display(Load_Lag7_test)

Scaled load - Test: 


array([[0.25466582, 0.29613534, 0.75384647, ..., 0.25714816, 0.3048297 ,
        0.38588761],
       [0.24051084, 0.28953229, 0.75899932, ..., 0.24841774, 0.29052281,
        0.38305951],
       [0.22435239, 0.27169718, 0.75843423, ..., 0.23657667, 0.28044529,
        0.38772706],
       ...,
       [0.25608669, 0.33648852, 0.99309584, ..., 0.22142303, 0.24270501,
        0.39183375],
       [0.23395411, 0.32725215, 0.98897613, ..., 0.2025681 , 0.21913098,
        0.37582581],
       [0.21205685, 0.31132498, 0.98803666, ..., 0.18779946, 0.200035  ,
        0.36044968]])

Scaled lag 1 load - Test: 


array([[0.30113248, 0.38478634, 0.81860239, ..., 0.38494814, 0.43840778,
        0.51463115],
       [0.28174071, 0.35459275, 0.81469837, ..., 0.35982849, 0.42598168,
        0.48850306],
       [0.24905883, 0.33206893, 0.80004491, ..., 0.32316076, 0.40192601,
        0.45290905],
       ...,
       [0.30339169, 0.39173253, 1.08009955, ..., 0.29042756, 0.28576307,
        0.49959415],
       [0.27932061, 0.36641505, 1.07479425, ..., 0.27567911, 0.26408997,
        0.47833474],
       [0.25046253, 0.33963488, 1.06205966, ..., 0.25559085, 0.23328951,
        0.46114322]])

Scaled lag 7 load - Test: 


array([[0.19931502, 0.22118697, 0.65766472, ..., 0.19380426, 0.21482035,
        0.31719445],
       [0.18429521, 0.22297768, 0.65582488, ..., 0.182058  , 0.19153506,
        0.30907862],
       [0.16303425, 0.20096888, 0.65502762, ..., 0.16297953, 0.17062645,
        0.2977502 ],
       ...,
       [0.22166433, 0.24499598, 0.9864243 , ..., 0.18879332, 0.21881339,
        0.30240937],
       [0.20106545, 0.23408521, 0.98424503, ..., 0.17812348, 0.20718886,
        0.29144344],
       [0.17693633, 0.20703807, 0.97846938, ..., 0.16221794, 0.19304775,
        0.27366191]])

In [13]:
## Transformation assigns perfectly.
Load_train[0]

array([0.19856039, 0.30747691, 0.29986234, 0.33139125, 0.3751176 ,
       0.29388401, 0.40075164, 0.54746011])

In [14]:
Load_Lag1_train[24]

array([0.19856039, 0.30747691, 0.29986234, 0.33139125, 0.3751176 ,
       0.29388401, 0.40075164, 0.54746011])

In [15]:
Load_Lag7_train[24*7]

array([0.19856039, 0.30747691, 0.29986234, 0.33139125, 0.3751176 ,
       0.29388401, 0.40075164, 0.54746011])

In [16]:
## Transformation assigns perfectly.
Load_test[5]

array([0.18947889, 0.26369694, 0.78863684, 0.23509279, 0.30304928,
       0.22726219, 0.29166311, 0.41771518])

In [17]:
Load_Lag1_test[24+5]

array([0.18947889, 0.26369694, 0.78863684, 0.23509279, 0.30304928,
       0.22726219, 0.29166311, 0.41771518])

In [18]:
Load_Lag7_test[24*7+5]

array([0.18947889, 0.26369694, 0.78863684, 0.23509279, 0.30304928,
       0.22726219, 0.29166311, 0.41771518])

## Processing the temperature data in train test split

In [19]:
# Fetching and concatenating the data
Temp_DS = pd.concat([pd.read_csv(os.path.join(path, fileName), usecols= ["Temprature"]) for fileName in fileList], axis= 1)
Temp_DS.columns = [i.removesuffix('.csv') for i in fileList]
Temp_DS = Temp_DS.add_prefix("TEMP_")

Temp_DS

,TEMP_COAST,TEMP_EAST,TEMP_FWEST,TEMP_NCENT,TEMP_NORTH,TEMP_SCENT,TEMP_SOUTH,TEMP_WEST
0,6.8,4.6,-5.9,4.6,-8.8,1.3,5.8,-7.1
1,7.1,4.6,-5.8,4.6,-8.8,0.8,5.7,-6.9
2,7.1,4.5,-5.8,4.5,-8.8,0.6,5.8,-6.5
3,7.0,4.3,-5.7,4.3,-8.7,0.8,5.7,-6.1
4,6.5,3.9,-5.7,3.9,-8.6,0.9,5.4,-5.7
...,...,...,...,...,...,...,...,...
43818,14.8,13.4,11.7,13.4,9.2,13.7,18.1,11.7
43819,14.3,13.8,12.3,13.8,9.9,14.2,17.8,12.2
43820,14.1,13.7,13.3,13.7,10.0,14.0,17.2,12.2
43821,13.7,12.5,13.2,12.5,9.2,12.9,16.6,11.6


In [20]:
# Scaling the temperature data for each individual region
mmScaler_temp = MinMaxScaler()

# Splitting the data into train and test
Temp_train, Temp_test = Temp_DS[:Num_train], Temp_DS[Num_train:]
print("Raw temperature - Train: ")
display(Temp_train.head(5))
print("Raw temperature - Test: ")
display(Temp_test.head(5))
print("\n")

# Scaling the data using mix-max scaler
Temp_train = mmScaler_temp.fit_transform(Temp_train)
print("Scaled temperature - Train: ")
display(Temp_train)

Temp_test = mmScaler_temp.transform(Temp_test)
print("Scaled temperature - Test: ")
display(Temp_test)

Raw temperature - Train: 


,TEMP_COAST,TEMP_EAST,TEMP_FWEST,TEMP_NCENT,TEMP_NORTH,TEMP_SCENT,TEMP_SOUTH,TEMP_WEST
0,6.8,4.6,-5.9,4.6,-8.8,1.3,5.8,-7.1
1,7.1,4.6,-5.8,4.6,-8.8,0.8,5.7,-6.9
2,7.1,4.5,-5.8,4.5,-8.8,0.6,5.8,-6.5
3,7.0,4.3,-5.7,4.3,-8.7,0.8,5.7,-6.1
4,6.5,3.9,-5.7,3.9,-8.6,0.9,5.4,-5.7


Raw temperature - Test: 


,TEMP_COAST,TEMP_EAST,TEMP_FWEST,TEMP_NCENT,TEMP_NORTH,TEMP_SCENT,TEMP_SOUTH,TEMP_WEST
35059,16.2,14.9,15.1,14.9,11.0,16.1,17.6,15.5
35060,15.5,14.9,15.7,14.9,11.2,16.1,17.2,15.5
35061,13.4,13.8,15.6,13.8,10.8,15.8,16.4,15.2
35062,12.0,11.6,14.3,11.6,7.6,13.5,14.8,13.3
35063,9.9,9.8,11.0,9.8,2.8,12.7,13.0,8.9




Scaled temperature - Train: 


array([[0.38445808, 0.31902985, 0.08630394, ..., 0.23163842, 0.19642857,
        0.15652174],
       [0.39059305, 0.31902985, 0.08818011, ..., 0.22222222, 0.19419643,
        0.16      ],
       [0.39059305, 0.31716418, 0.08818011, ..., 0.21845574, 0.19642857,
        0.16695652],
       ...,
       [0.48261759, 0.42910448, 0.41463415, ..., 0.41996234, 0.36830357,
        0.49217391],
       [0.53169734, 0.46641791, 0.44277674, ..., 0.46516008, 0.39955357,
        0.51478261],
       [0.55419223, 0.50373134, 0.46529081, ..., 0.4952919 , 0.44196429,
        0.53565217]])

Scaled temperature - Test: 


array([[0.57668712, 0.51119403, 0.48030019, ..., 0.51035782, 0.45982143,
        0.54956522],
       [0.56237219, 0.51119403, 0.49155722, ..., 0.51035782, 0.45089286,
        0.54956522],
       [0.5194274 , 0.49067164, 0.48968105, ..., 0.5047081 , 0.43303571,
        0.54434783],
       ...,
       [0.53374233, 0.48880597, 0.44652908, ..., 0.47080979, 0.45089286,
        0.49217391],
       [0.52556237, 0.46641791, 0.44465291, ..., 0.45009416, 0.4375    ,
        0.48173913],
       [0.50920245, 0.40298507, 0.4108818 , ..., 0.41619586, 0.421875  ,
        0.44695652]])

# Preparing the training and testing split for model input

In [21]:
X_train = np.array([[[i, j, k] for i,j,k in zip(Temp_train[m], Load_Lag1_train[m], Load_Lag7_train[m])] for m in range(len(Temp_train))])
print(X_train.shape)
print(X_train[0])

(35059, 8, 3)
[[ 0.38445808 -0.55818217 -0.55818217]
 [ 0.31902985 -0.42906604 -0.42906604]
 [ 0.08630394 -0.62275662 -0.62275662]
 [ 0.31902985 -0.37781528 -0.37781528]
 [ 0.18019481 -0.5336438  -0.5336438 ]
 [ 0.23163842 -0.34536602 -0.34536602]
 [ 0.19642857 -0.4410945  -0.4410945 ]
 [ 0.15652174 -0.53876871 -0.53876871]]


In [22]:
X_test = np.array([[[i, j, k] for i,j,k in zip(Temp_test[m], Load_Lag1_test[m], Load_Lag7_test[m])] for m in range(len(Temp_test))])
print(X_test.shape)
print(X_test[0])

(8764, 8, 3)
[[0.57668712 0.30113248 0.19931502]
 [0.51119403 0.38478634 0.22118697]
 [0.48030019 0.81860239 0.65766472]
 [0.51119403 0.396774   0.2219211 ]
 [0.50162338 0.41072623 0.23564975]
 [0.51035782 0.38494814 0.19380426]
 [0.45982143 0.43840778 0.21482035]
 [0.54956522 0.51463115 0.31719445]]


In [23]:
# Setting up the batch and node parameters
num_batch, num_node = Load_DS.shape

number_feat = X_train.shape[2]

print("No. of batches: ", num_batch)
print("No. of nodes: " , num_node)
print("No. of features: ", number_feat)

No. of batches:  43823
No. of nodes:  8
No. of features:  3


In [24]:
edge_index = torch.LongTensor([np.repeat(range(num_node), num_node-1).tolist(),
                               [j for i in range(num_node) for j in range(num_node) if i != j]])

X_train, X_test = Variable(torch.FloatTensor(X_train)), torch.FloatTensor(X_test)
Y_train, Y_test = Variable(torch.FloatTensor(Load_train)), Load_test

In [25]:
edge_index

tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3,
         3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6,
         6, 7, 7, 7, 7, 7, 7, 7],
        [1, 2, 3, 4, 5, 6, 7, 0, 2, 3, 4, 5, 6, 7, 0, 1, 3, 4, 5, 6, 7, 0, 1, 2,
         4, 5, 6, 7, 0, 1, 2, 3, 5, 6, 7, 0, 1, 2, 3, 4, 6, 7, 0, 1, 2, 3, 4, 5,
         7, 0, 1, 2, 3, 4, 5, 6]])

## DEFINING THE MODEL

In [26]:
torch.manual_seed(666)

model = Net(number_feat, edge_index, num_node)

optimizer = torch.optim.Adam([
    dict(params=model.conv1.parameters(), weight_decay=5e-4),
    dict(params=model.conv2.parameters(), weight_decay=0),
    dict(params=model.conv3.parameters(), weight_decay=0),
], lr=0.01) 

In [27]:
Epochs = 100

dataset = TensorDataset(X_train, Y_train)    
loader = DataLoader(dataset, batch_size = 64, shuffle=False)  

In [28]:
train_start = time.time()

In [29]:
model.train()

for epoch in range(Epochs):
    loss = 0
    for step, (x, y) in enumerate(loader):
        optimizer.zero_grad()
        loss_train = 0
        for i in range(x.size(0)):
            output = model(x[i])
            loss_train += F.l1_loss(output[0][0], y[i,0]) + F.l1_loss(output[1][0], y[i,1]) + F.l1_loss(output[2][0], y[i,2]) + F.l1_loss(output[3][0], y[i,3]) + F.l1_loss(output[4][0], y[i,4]) + F.l1_loss(output[5][0], y[i,5]) + F.l1_loss(output[6][0], y[i,6]) + F.l1_loss(output[7][0], y[i,7]) 
        loss += loss_train/i
        loss_train.backward()
        optimizer.step()
    loss = loss/step
    print( "Epoch {}: the train loss = {:.4f}".format(epoch+1, loss))

Epoch 1: the train loss = 1.7603
Epoch 2: the train loss = 1.4988
Epoch 3: the train loss = 1.4918
Epoch 4: the train loss = 1.4908
Epoch 5: the train loss = 1.4871
Epoch 6: the train loss = 1.4860
Epoch 7: the train loss = 1.4870
Epoch 8: the train loss = 1.4830
Epoch 9: the train loss = 1.4828
Epoch 10: the train loss = 1.4816
Epoch 11: the train loss = 1.4813
Epoch 12: the train loss = 1.4812
Epoch 13: the train loss = 1.4814
Epoch 14: the train loss = 1.4805
Epoch 15: the train loss = 1.4795
Epoch 16: the train loss = 1.4784
Epoch 17: the train loss = 1.4799
Epoch 18: the train loss = 1.4793
Epoch 19: the train loss = 1.4782
Epoch 20: the train loss = 1.4773
Epoch 21: the train loss = 1.4792
Epoch 22: the train loss = 1.4802
Epoch 23: the train loss = 1.4787
Epoch 24: the train loss = 1.4785
Epoch 25: the train loss = 1.4771
Epoch 26: the train loss = 1.4774
Epoch 27: the train loss = 1.4792
Epoch 28: the train loss = 1.4771
Epoch 29: the train loss = 1.4772
Epoch 30: the train los

In [30]:
train_end = time.time()

In [31]:
print("Training time: ", train_end - train_start)

Training time:  11652.486827373505


## MAKING PREDICTIONS

In [32]:
test_start = time.time()

In [33]:
predictions = []

# Switching to eval mode
model.eval()

with torch.no_grad():
    for i in range(X_test.size(0)):
        p = model(X_test[i])
        predictions.append(torch.cat([p[0], p[1], p[2], p[3], p[4], p[5], p[6], p[7]], dim=-1).tolist())

predictions = np.array(predictions)

In [34]:
test_end = time.time()

In [35]:
print("Testing time: ", test_end - test_start)

Testing time:  7.785362482070923


In [36]:
predictions

array([[0.21697968, 0.40198055, 0.0279208 , ..., 0.2590788 , 0.35938683,
        0.75087082],
       [0.20366171, 0.39539388, 0.02377558, ..., 0.25263041, 0.34726146,
        0.73948097],
       [0.19602028, 0.39161465, 0.02139723, ..., 0.24893057, 0.34030446,
        0.7329458 ],
       ...,
       [0.20684403, 0.39696774, 0.02476607, ..., 0.25417128, 0.35015884,
        0.74220252],
       [0.19866976, 0.39292505, 0.02222191, ..., 0.25021338, 0.3427166 ,
        0.73521185],
       [0.16936705, 0.38237759, 0.01544304, ..., 0.23655456, 0.31744668,
        0.71338737]])

In [37]:
inversed_predictions = mmScaler_load.inverse_transform(predictions)
inversed_predictions

array([[10083.81048959,  1524.0998543 ,  1465.55272056, ...,
         6364.80282781,  3430.62713213,  1746.2442267 ],
       [ 9910.56162602,  1512.02021364,  1456.21623806, ...,
         6296.9013136 ,  3378.66135666,  1730.82172918],
       [ 9811.1568645 ,  1505.08928124,  1450.85937581, ...,
         6257.94201487,  3348.84572795,  1721.97273931],
       ...,
       [ 9951.9593415 ,  1514.9065991 ,  1458.44717881, ...,
         6313.12666618,  3391.07868127,  1734.50685784],
       [ 9845.6231057 ,  1507.49250731,  1452.71683175, ...,
         6271.45001396,  3359.18345084,  1725.04109721],
       [ 9464.4342783 ,  1488.14896966,  1437.44845867, ...,
         6127.62264912,  3250.88399547,  1695.48952906]])

In [38]:
inversed_Y_test = mmScaler_load.inverse_transform(Y_test)
inversed_Y_test

array([[10574.056358,  1329.984791,  3100.590775, ...,  6344.473191,
         3196.811331,  1252.0365  ],
       [10389.919171,  1317.875105,  3112.196787, ...,  6252.541918,
         3135.496237,  1248.207087],
       [10179.719448,  1285.166362,  3110.924014, ...,  6127.855412,
         3092.306962,  1254.527211],
       ...,
       [10592.539874,  1403.990584,  3639.463899, ...,  5968.287603,
         2930.56323 ,  1260.087906],
       [10304.62487 ,  1387.051521,  3630.184859, ...,  5769.74522 ,
         2829.531899,  1238.412254],
       [10019.771129,  1357.841863,  3628.068853, ...,  5614.231555,
         2747.692143,  1217.592105]])

In [39]:
MAPE1 = mape(inversed_Y_test[:, 0].reshape(-1), inversed_predictions[:, 0].reshape(-1))
MAPE2 = mape(inversed_Y_test[:, 1].reshape(-1), inversed_predictions[:, 1].reshape(-1))
MAPE3 = mape(inversed_Y_test[:, 2].reshape(-1), inversed_predictions[:, 2].reshape(-1))
MAPE4 = mape(inversed_Y_test[:, 3].reshape(-1), inversed_predictions[:, 3].reshape(-1))
MAPE5 = mape(inversed_Y_test[:, 4].reshape(-1), inversed_predictions[:, 4].reshape(-1))
MAPE6 = mape(inversed_Y_test[:, 5].reshape(-1), inversed_predictions[:, 5].reshape(-1))
MAPE7 = mape(inversed_Y_test[:, 6].reshape(-1), inversed_predictions[:, 6].reshape(-1))
MAPE8 = mape(inversed_Y_test[:, 7].reshape(-1), inversed_predictions[:, 7].reshape(-1))

print( "The test mape is {:.3f}, {:.3f}, {:.3f}, {:.3f}, {:.3f}, {:.3f}, {:.3f}, {:.3f}".format(MAPE1, MAPE2, MAPE3, MAPE4, MAPE5, MAPE6, MAPE7, MAPE8))

The test mape is 10.088, 6.791, 45.232, 4.339, 13.506, 6.404, 4.864, 22.671


#### AVERAGE MAPE

In [40]:
meanMAPE = np.mean([MAPE1, MAPE2, MAPE3, MAPE4, MAPE5, MAPE6, MAPE7, MAPE8])
meanMAPE

14.23689920968907

#### WEIGHTED MAPE

In [41]:
# Scaled MAPE with max values
weightMax = pd.Series(Load_DS.max() / sum(Load_DS.max())).reset_index(drop= True)
print(weightMax)
print("\n")
mapeMetric = pd.Series([MAPE1, MAPE2, MAPE3, MAPE4, MAPE5, MAPE6, MAPE7, MAPE8])
print(mapeMetric)
print("\n")

weightedMAPE = weightMax.multiply(mapeMetric)
weightedMAPE = sum(weightedMAPE)
print("Weighted using the max load of a region: ", weightedMAPE)

0    0.270208
1    0.033316
2    0.054758
3    0.336858
4    0.019346
5    0.180087
6    0.078511
7    0.026916
dtype: float64


0    10.088311
1     6.790699
2    45.232125
3     4.339156
4    13.505687
5     6.403539
6     4.864367
7    22.671311
dtype: float64


Weighted using the max load of a region:  9.297272464052263


In [42]:
# Scaled MAPE with average values
weightMax = pd.Series(Load_DS.mean() / sum(Load_DS.mean())).reset_index(drop= True)
print(weightMax)
print("\n")
mapeMetric = pd.Series([MAPE1, MAPE2, MAPE3, MAPE4, MAPE5, MAPE6, MAPE7, MAPE8])
print(mapeMetric)
print("\n")

weightedMAPE = weightMax.multiply(mapeMetric)
weightedMAPE = sum(weightedMAPE)
print("Weighted using the mean load of a region: ", weightedMAPE)

0    0.287236
1    0.034634
2    0.062310
3    0.322066
4    0.020268
5    0.161189
6    0.083478
7    0.028818
dtype: float64


0    10.088311
1     6.790699
2    45.232125
3     4.339156
4    13.505687
5     6.403539
6     4.864367
7    22.671311
dtype: float64


Weighted using the mean load of a region:  9.71415853698876
